In [40]:
import cv2
from tracker import *
import numpy as np
from matplotlib import pyplot as plt
import imutils 


In [55]:
# Create tracker object
tracker = EuclideanDistTracker()

cap = cv2.VideoCapture("video/Stereo conveyor with occlusions.mp4")

# Object detection from Stable camera
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)
k=0
while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape

    # Extract Region of interest
    #frame = frame[300:900,400:1150]
    #rotated = imutils.rotate_bound(frame, 15)
    #rotated2=rotated[450:800,350:1250]
    #roi = imutils.rotate_bound(rotated2, -15)
    roi = frame
    
    # 1. Object Detection
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 20, 255, cv2.THRESH_BINARY)
    thresh = cv2.dilate(mask, None, iterations=6)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []

    biggest_area=0
    best_cnt = None
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area>biggest_area:
            biggest_area=area
            best_cnt = cnt
    if (biggest_area>4500 and biggest_area<76000):
        x, y, w, h = cv2.boundingRect(best_cnt)
        detections.append([x, y, w, h])
    # 2. Object Tracking
    boxes_ids = tracker.update(detections)
    c=False
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        rx=x-25
        ry=y-25
        rw=w+50
        rh=h+50
        centerx=(rx+rx+rw)//2
        centery=(ry+ry+rh)//2
        c=cv2.rectangle(roi, (rx, ry), (rx + rw, ry + rh), (0, 255, 0), 3)
        cv2.circle(roi,(centerx,centery),5,(0,0,255), -1)  
        #cv2.rectangle(mask, (x, y), (x + w, y + h), (0, 255, 0), 3)
        cv2.rectangle(frame, (rx, ry), (rx + rw, ry + rh), (0, 255, 0), 3)
        cv2.circle(frame,(centerx,centery),5,(0,0,255), -1)      
        print('x',x)
        print('y',y)
        if x>0 and y>0:
            crop_img = roi[y:y+h, x:x+w]
            ncrop_img=cv2.resize(crop_img, (128, 128), interpolation = cv2.INTER_AREA)
            k+=1
            cv2.imwrite('video_crop/'+str(k)+'.jpg', ncrop_img)

    cv2.imshow("roi", roi)
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", thresh)

    key = cv2.waitKey(30) # can be removed 
    if cv2.waitKey(1) & 0xFF == ord('q'):  # q will end the program
        break

cap.release()
cv2.destroyAllWindows()

x 872
y 141
{0: (940, 169)}
x 872
y 140
{0: (939, 169)}
x 870
y 141
x 440
y 0
x 728
y 0
x 869
y 125
{3: (940, 168)}
x 871
y 139
x 48
y 229
x 871
y 125
{5: (951, 161)}
x 871
y 125
{5: (940, 169)}
x 871
y 140
{5: (940, 169)}
x 871
y 140
{5: (940, 169)}
x 871
y 140
x 872
y 139
{6: (940, 170)}
x 871
y 139
{6: (940, 170)}
x 871
y 139
x 1223
y 519
{7: (1240, 581)}
x 1201
y 508
x 721
y 551
x 722
y 466
x 769
y 292
{10: (912, 466)}
x 778
y 271
x 803
y 246
{11: (942, 442)}
x 830
y 246
{11: (950, 432)}
x 843
y 236
{11: (963, 423)}
x 865
y 223
{11: (973, 418)}
x 884
y 223
{11: (984, 407)}
x 905
y 219
{11: (998, 426)}
x 929
y 244
x 943
y 372
{12: (1013, 471)}
x 954
y 372
x 1217
y 328
{13: (1241, 363)}
x 1202
y 314
{13: (1233, 350)}
x 1187
y 300
{13: (1226, 342)}
x 1173
y 288
{13: (1219, 332)}
x 1158
y 277
{13: (1213, 325)}
x 1146
y 270
{13: (1201, 320)}
x 1123
y 265
x 1054
y 261
{14: (1169, 343)}
x 1059
y 258
{14: (1175, 342)}
x 1070
y 255
{14: (1177, 341)}
x 1075
y 253
{14: (1180, 335)}
x 1080
y 2

In [5]:
# VERSION 1
import cv2
cap = cv2.VideoCapture("Stereo conveyor without occlusions.mp4")
ret, frame = cap.read()
firstFrame = None
while ret:
    #Detecting movement
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    rotated = imutils.rotate_bound(gray, 15)
    rotated2=rotated[450:800,350:1250]
    roi = imutils.rotate_bound(rotated2, -15)
    if firstFrame is None:
        firstFrame = roi
        continue

    feat1 = cv2.goodFeaturesToTrack(roi, maxCorners=1000, qualityLevel=0.4, minDistance=10)
    feat2, status, error = cv2.calcOpticalFlowPyrLK(roi, firstFrame, feat1, None)

    for i in range(len(feat1)):
        f10=int(feat1[i][0][0])
        f11=int(feat1[i][0][1])
        f20=int(feat2[i][0][0])
        f21=int(feat2[i][0][1])
        #cv2.circle(frame, (f10, f11), 5, (0, 255, 0), -1)
        #cv2.line(frame, (f10,f11), (f20, f21), (0, 255, 0), 2)

        if ((abs(f10-f20))/(f10+f20) >= 0.05) or ((abs(f11-f21))/(f11+f21) >= 0.05): # 0.005
            
            #cv2.line(frame, (f10,f11), (f20, f21), (0, 255, 0), 2)
            cv2.circle(roi, (f10, f11), 5, (0, 255, 0), -1)
    cv2.imshow('image', roi)
    cv2.waitKey
    
    #firstFrame = gray
    ret, frame = cap.read()
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyAllWindows()
cap.release()